In [1]:
import os
from pathlib import Path
import pandas as pd
from collections import defaultdict
from utils.get_processed_dataset import get_processed_dataset
from utils.utils import read_jsonl, get_major_entities
from omegaconf import OmegaConf
from configs.config import PRONOUNS_GROUPS, PLURAL_PRONOUNS, dataset_yaml, selected_keys, entity_gender_metadata, pronoun_dialogue_metadata
from configs.config_gen import NAME_TO_PREFIX
from tqdm.auto import tqdm
import hydra
from utils.qa_utils import write_qa_to_jsonl, get_mention_info, add_copelands_count
import dtale
import jsonlines
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from hydra.core.global_hydra import GlobalHydra
from openai import OpenAI
from dotenv import load_dotenv

/home/admin/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.chdir(Path(__file__).resolve().parents[1])

In [2]:
input_format_parts = ["{{", "}} (#This is the marked mention)"]
output_format_parts = ["- Mention: ", "- Explanation:", "- The mention refers to:"]

In [3]:
def read_outputs_results(args):
    ### Read the output file and result file
    output_file = args.paths["output_file"]
    result_file = args.paths["result_file"]
    output_list = []
    with jsonlines.open(output_file) as reader:
        output_list = [obj for obj in reader]
    result_str = open(result_file).read()
    return output_list, result_str

def get_entity_pronoun(dataset_proc_all, major_entities_all, doc_key):
    pronoun_info = {}
    for ind,entity_id in enumerate(major_entities_all[doc_key]["entity_id"]):
        entity_mentions = dataset_proc_all[doc_key]["clusters_vs_mentions"][entity_id]
        entity_mentions_str = [dataset_proc_all[doc_key]["mentions_vs_mentionstr"][mention].lower() for mention in entity_mentions]
        entity_mentions_pro_grps = [PRONOUNS_GROUPS[mention] for mention in entity_mentions_str if mention in PRONOUNS_GROUPS]
        counter_pro_grps = Counter(entity_mentions_pro_grps)
        ## Convert to a set ordered by frequency of grps
        counter_pro_grps = {k: v for k, v in sorted(counter_pro_grps.items(), key=lambda item: item[1], reverse=True)}
        pronoun_info[major_entities_all[doc_key]["entity_name"][ind]] = counter_pro_grps
    return pronoun_info

In [4]:
dataset_configs = OmegaConf.load(dataset_yaml)
mention_info_df_final = pd.DataFrame()
major_entities_all = {}
dataset_proc_all = {}
gender_metadata = pd.read_csv(entity_gender_metadata)
pron_dialogue_metadata = pd.read_csv(pronoun_dialogue_metadata)
for dataset_name in ["litbank", "fantasy"]:
    dataset_source = dataset_configs[dataset_name][f"train_file"]
    tsv_addr = dataset_configs[dataset_name][f"tsv"]
    doc_me = dataset_configs[dataset_name][f"train_me"]
    dataset = read_jsonl(dataset_source)
    dataset_proc = get_processed_dataset(dataset, tsv_litbank=tsv_addr)
    dataset_proc_all.update(dataset_proc)
    major_entities = get_major_entities(doc_me)
    major_entities_all.update(major_entities)
    mention_info_df = get_mention_info(dataset_proc, major_entities, tsv_addr=tsv_addr)
    mention_info_df["dataset"] = dataset_name
    mention_info_df_final = pd.concat([mention_info_df_final, mention_info_df])

/home/admin/miniconda3/envs/myenv/lib/python3.12/site-packages/thinc/shims/pytorch.py:253: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



599_vanity_fair_brat_0
155_the_moonstone_brat_0
2641_a_room_with_a_view_brat_0
1245_night_and_day_brat_0
2891_howards_end_brat_0
2166_king_solomons_mines_brat_0
3457_the_man_of_the_forest_brat_0
932_the_fall_of_the_house_of_usher_brat_0
2084_the_way_of_all_flesh_brat_0
514_little_women_brat_0
120_treasure_island_brat_0
2005_piccadilly_jim_brat_0
27_far_from_the_madding_crowd_brat_0
62_a_princess_of_mars_brat_0
74_the_adventures_of_tom_sawyer_brat_0
36_the_war_of_the_worlds_brat_0
11_alices_adventures_in_wonderland_brat_0
432_the_ambassadors_brat_0
24_o_pioneers_brat_0
32_herland_brat_0
2775_the_good_soldier_brat_0
5348_ragged_dick_or_street_life_in_new_york_with_the_bootblacks_brat_0
9830_the_beautiful_and_damned_brat_0
78_tarzan_of_the_apes_brat_0
1064_the_masque_of_the_red_death_brat_0
209_the_turn_of_the_screw_brat_0
41_the_legend_of_sleepy_hollow_brat_0
1400_great_expectations_brat_0
217_sons_and_lovers_brat_0
3268_the_mysteries_of_udolpho_brat_0
73_the_red_badge_of_courage_an_epis

/home/admin/miniconda3/envs/myenv/lib/python3.12/site-packages/thinc/shims/pytorch.py:253: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



124_The_Three_Brothers_0
007_The_Good_Bargain_0
077_Clever_Grethel_0
135_The_White_Bride_and_the_Black_One_0
083_Hans_in_Luck_0
044_Godfather_Death_0
016_The_Three_Snake-Leaves_0
049_The_Six_Swans_0
193_The_Drummer_0
157_The_Sparrow_And_His_Four_Children_0
142_Simeli_Mountain_0
103_Sweet_Porridge_0
009_The_Twelve_Brothers_0
211_The_Twelve_Idle_Servants_0
006_Faithful_John_0
112_The_Flail_From_Heaven_0
176_The_Duration_of_Life_0
184_The_Nail_0
060_The_Two_Brothers_0
105_Stories_about_Snakes_0
037_Thumbling_0
153_The_Star-Money_0
113_The_Two_Kings’_Children_0
137_The_Three_Black_Princesses_0
210_The_Hazel-Branch_0
173_The_Bittern_and_the_Hoopoe_0
141_The_Lambkin_and_the_Little_Fish_0
131_Fair_Katrinelje_and_Pif-Paf-Poltrie_0
192_The_Master-Thief_0
090_The_Young_Giant_0
097_The_Water_of_Life_0
159_The_Ditmarsch_Tale_of_Wonders_0
086_The_Fox_and_the_Geese_0
003_Our_Lady’s_Child_0
132_The_Fox_and_the_Horse_0
119_The_Seven_Swabians_0
053_Little_Snow-white_0
188_The_Spindle,_The_Shuttle,_and_

In [5]:
def get_accuracy_dict(mention_info_filt):
    
    categories = {
    "overall": {},
    "NOM": {"filter": mention_info_filt["category"] == "NOM"},
    "PRON": {"filter": mention_info_filt["category"] == "PRON"},
    "fan_NOM": {"filter": (mention_info_filt["dataset"] == "fantasy") & (mention_info_filt["category"] == "NOM")},
    "fan_PRON": {"filter": (mention_info_filt["dataset"] == "fantasy") & (mention_info_filt["category"] == "PRON")},
    "lit_NOM": {"filter": (mention_info_filt["dataset"] == "litbank") & (mention_info_filt["category"] == "NOM")},
    "lit_PRON": {"filter": (mention_info_filt["dataset"] == "litbank") & (mention_info_filt["category"] == "PRON")},
    "opts": {"filter": mention_info_filt["answer"] != "None of the Above"},
    "noa": {"filter": mention_info_filt["answer"] == "None of the Above"},
    "nested": {"filter": mention_info_filt["is_nested"] == True},
    "unnested": {"filter": mention_info_filt["is_nested"] == False},
    "male": {"filter": mention_info_filt["ans_gender"] == "Male"},
    "female": {"filter": mention_info_filt["ans_gender"] == "Female"},
    "neutral": {"filter": mention_info_filt["ans_gender"] == "Neutral"},
    "pers_pron": {"filter": mention_info_filt["pronoun_class"].isin([0, 1])},
    "othr_pron": {"filter": mention_info_filt["pronoun_class"].isin([2, 3, 4, 7])},
    "dialogue": {"filter": (mention_info_filt["category"] == "PRON") & (mention_info_filt["is_dialogue"] == True)},
    "non_dialogue": {"filter": (mention_info_filt["category"] == "PRON") & (mention_info_filt["is_dialogue"] == False)}
}
    for key, val in categories.items():
        filt_data = mention_info_filt[val.get("filter", slice(None))]
        count = filt_data.shape[0]
        mean = filt_data["correct"].mean() * 100
        categories[key].update({"count": count, "mean": mean})
    
    accuracy_dict = {}
    for key in categories:
        accuracy_dict[key] = categories[key]["mean"]
    
    count_dict = {}
    for key in categories:
        count_dict[key] = categories[key]["count"]

    for key in accuracy_dict:
        accuracy_dict[key] = round(accuracy_dict[key], 2)
    return accuracy_dict, count_dict

In [6]:
MODEL_TO_COT = {
    "gemini-1.5-flash": "desc",
    "gpt-4o-mini": "desc",
    "gpt-4o": "desc",
    "mistralai/Mistral-7B-Instruct-v0.3": "desc",
    "meta-llama/Meta-Llama-3.1-8B-Instruct": "desc"
}

In [7]:
def get_hydra_config(model_name):
    # Clear the previous Hydra instance if it's already initialized
    if GlobalHydra.instance().is_initialized():
        GlobalHydra.instance().clear()
    hydra.initialize(config_path="configs/args")
    args = hydra.compose(config_name="args_qa.yaml")
    args["model_name"] = model_name
    args["model_prefix"] = NAME_TO_PREFIX.get(args.model_name, args.model_name[:5])
    if args.precision != 16:
        args["model_prefix"] = f"{args.model_prefix}_precision_{args.precision}"
    args["cot"] = MODEL_TO_COT[model_name]
    for key in args.paths:
        args.paths[key] = Path(args.paths[key])
    print(args)
    return args

In [8]:
def get_nested_info(mention_info_df):
    nested_info = {}
    nested_info["nested_count"] = mention_info_df[mention_info_df["is_nested"] == True].shape[0]
    nested_info["nested_errors"] = mention_info_df[(mention_info_df["is_nested"] == True) & (mention_info_df["correct"] == False)].shape[0]
    nested_info["nested_accuracy"] = round((1 - nested_info["nested_errors"]/nested_info["nested_count"]) * 100, 2)
    nested_errors_df = mention_info_df[(mention_info_df["is_nested"] == True) & (mention_info_df["correct"] == False)]
    in_nested_count = 0
    for ind, row in nested_errors_df.iterrows():
        if row["predicted_answer"] in row["nested_entities"]:
            in_nested_count += 1
    nested_info["in_nested_count"] = in_nested_count
    nested_info["in_nested_perc_error"] = round((in_nested_count/nested_info["nested_errors"]) * 100, 2)
    nested_info["in_nested_perc_total"] = round((in_nested_count/nested_info["nested_count"]) * 100, 2)
    return nested_info
    

In [9]:
def get_option_info(mention_info_df):
    option_info = {}
    option_info["Option - Option"] = mention_info_df[(mention_info_df["answer"] != "None of the Above") & (mention_info_df["predicted_answer"] != "None of the Above")].shape[0]
    option_info["Option - NoA"] = mention_info_df[(mention_info_df["answer"] != "None of the Above") & (mention_info_df["predicted_answer"] == "None of the Above")].shape[0]
    option_info["NoA - Option"] = mention_info_df[(mention_info_df["answer"] == "None of the Above") & (mention_info_df["predicted_answer"] != "None of the Above")].shape[0]
    option_info["NoA - NoA"] = mention_info_df[(mention_info_df["answer"] == "None of the Above") & (mention_info_df["predicted_answer"] == "None of the Above")].shape[0]
    return option_info

In [10]:
def get_gpt_gender_output(entity_name, doc_key):
    load_dotenv()
    prompt = f"""Classify the given phrase based on its gender. If the phrase clearly indicates a male name or title, classify it as 'Male.' If the phrase indicates a female name or title, classify it as 'Female.' If the phrase does not specify a male or female name/title, or it is ambiguous, classify it as 'Neutral.' The phrase may include titles, names, or other identifiers. Your options are: Male, Female, Neutral.

Follow the format below:
Explanation: 
Gender: 

The phrase is: \"\"\"{entity_name}\"\"\""""
    ## Create an api call to gpt-4o-mini
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
    conversation = [{"role": "user", "content": prompt},]
    output_string = "" 
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=conversation,
            temperature=0.0,
            max_tokens=4095,
            stream=True,
        )

        for chunk in completion:
            if chunk.choices[0].delta.content != None:
                output_string += chunk.choices[0].delta.content
            else:
                finish_reason = chunk.choices[0].finish_reason

    except Exception as e:
        print(f"Error in Document ID: {doc_key} and Entity Name: {entity_name}")
        print("Error: ", e)
        return {}

    if finish_reason == "content_filter":
        print("Content Filter Error in Document ID: {doc_key} and Entity Name: {entity_name}")
        print(f"Content Filter Error: {completion}")
        return {}

    if output_string != "":
        predicted_answer = output_string.split("Gender:")[-1].strip()
        print(output_string)
        return {
            "doc_key": doc_key,
            "entity_name": entity_name,
            "gender": predicted_answer,
            "prompted": True,
            "output_string": output_string
        }
    
    return {}


In [11]:
def get_gender_confusion(mention_info_df):
    options = ["Male", "Female", "Neutral", "None"]
    gender_dict = {}
    ## Get the confusion matrix
    for actual in options:
        for predicted in options:
            confusion_count = mention_info_df[(mention_info_df["ans_gender"] == actual) & (mention_info_df["pred_gender"] == predicted)].shape[0]
            gender_dict[f"{actual} - {predicted}"] = confusion_count
    return gender_dict

In [12]:
model_lists = ["mistralai/Mistral-7B-Instruct-v0.3", "meta-llama/Meta-Llama-3.1-8B-Instruct", "gpt-4o-mini", "gemini-1.5-flash", "gpt-4o"]
# model_lists = ["gpt-4o-mini"]
accuracy_dict = {}
count_dict = {}
nested_dict = {}
options_dict = {}
gender_dict = {}
gender_unnested_dict = {}
for model_name in model_lists:
    print(f"Processing model: {model_name}")
    args = get_hydra_config(model_name)

    ### Read the output file and result file
    output_list, result_str = read_outputs_results(args)
    mention_info_filt = []
    for obj in output_list:
        options = obj["options"]
        doc_key = obj["doc_key"]
        mention_ind = obj["mention_ind"]
        category = obj["category"]
        output_text = obj["output"]
        predicted_answer = obj["predicted_answer"]
        answer = obj["answer"]

        phrase_extr = (
                output_text.split(output_format_parts[0])[-1]
                .strip()
                .split(output_format_parts[1])[0]
                .strip()
                .replace("{", "")
                .replace("}", "")
            )
        

        ## Add dict corresponding to row in mention_info_df_final with same doc_key and mention_ind
        ques_dict = mention_info_df_final[(mention_info_df_final["doc_key"] == doc_key) & (mention_info_df_final["mention_ind"] == mention_ind)].to_dict(orient="records")[0]
        
        pronoun_info = get_entity_pronoun(dataset_proc_all, major_entities_all, doc_key)
        
        if predicted_answer != "None of the Above":
            if predicted_answer in options: 
                predicted_gender = gender_metadata[(gender_metadata["doc_key"] == doc_key) & (gender_metadata["entity_name"] == predicted_answer)]["gender"].values[0]
            else:
                gpt_output = get_gpt_gender_output(predicted_answer, doc_key)
                predicted_gender = gpt_output["gender"]
                # predicted_gender = "None"
        else:
            predicted_gender = "None"
        
        if answer != "None of the Above":
            ans_gender = gender_metadata[(gender_metadata["doc_key"] == doc_key) & (gender_metadata["entity_name"] == answer)]["gender"].values[0]
        else:
            ans_gender = "None"

        if category == "PRON":
            is_dialogue = pron_dialogue_metadata[(pron_dialogue_metadata["doc_key"] == doc_key) & (pron_dialogue_metadata["mention_ind"] == mention_ind)]["is_dialogue"].values[0]
        else:
            is_dialogue = False

        ques_dict.update(
            {
                "predicted_answer": predicted_answer,
                "answer": answer,
                "phrase_extr": phrase_extr,
                "correct": predicted_answer == answer,
                "options": options,
                "category": category,
                "pred_in_opts": predicted_answer in options,
                "pred_gender": predicted_gender,
                "ans_gender": ans_gender,
                "output": output_text,
                "predicted_pronoun": pronoun_info.get(predicted_answer, {}),
                "answer_pronoun": pronoun_info.get(answer, {}),
                "num_options": len(options),
                "is_dialogue": is_dialogue
            }
        )
        
        mention_info_filt.append(ques_dict)
    
    mention_info_filt = pd.DataFrame(mention_info_filt)
    mention_info_filt["correct_extr"] = mention_info_filt["mentions_vs_mentionstr"].str.strip() == mention_info_filt["phrase_extr"].str.strip()

    accuracy_dict[model_name], count_dict[model_name] = get_accuracy_dict(mention_info_filt)
    nested_dict[model_name] = get_nested_info(mention_info_filt)
    options_dict[model_name] = get_option_info(mention_info_filt[mention_info_filt["correct"] == False])
    gender_dict[model_name] = get_gender_confusion(mention_info_filt[mention_info_filt["correct"] == False])
    gender_unnested_dict[model_name] = get_gender_confusion(mention_info_filt[(mention_info_filt["correct"] == False) & (mention_info_filt["is_nested"] == False)].copy())
    mention_info_filt.to_csv(f"analysis_data/info/{args.model_prefix}_mention_info.csv", index=False)
    # dtale.show(mention_info_filt,subprocess=False, host='localhost')


Processing model: mistralai/Mistral-7B-Instruct-v0.3
{'model_name': 'mistralai/Mistral-7B-Instruct-v0.3', 'model_prefix': 'mistral_7b_instruct', 'precision': 16, 'split': 'test', 'cot': 'desc', 'naming_format': 'name', 'device': 'cuda:0', 'no_wandb': True, 'paths': {'data_root': PosixPath('data'), 'resource_root': PosixPath('qas'), 'result_root': PosixPath('results/qa'), 'instruction_file': PosixPath('data/qas/prompts/desc/instruction.txt'), 'format_file': PosixPath('data/qas/prompts/desc/format.txt'), 'eval_file': PosixPath('data/qas/data/qas_test.jsonl'), 'output_file': PosixPath('results/qa/mistral_7b_instruct/desc/output_test.jsonl'), 'result_file': PosixPath('results/qa/mistral_7b_instruct/desc/result_test.txt')}}


/tmp/ipykernel_1206751/1095863153.py:5: UserWarning:


The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1



Processing model: meta-llama/Meta-Llama-3.1-8B-Instruct
{'model_name': 'meta-llama/Meta-Llama-3.1-8B-Instruct', 'model_prefix': 'llama3_instruct', 'precision': 16, 'split': 'test', 'cot': 'desc', 'naming_format': 'name', 'device': 'cuda:0', 'no_wandb': True, 'paths': {'data_root': PosixPath('data'), 'resource_root': PosixPath('qas'), 'result_root': PosixPath('results/qa'), 'instruction_file': PosixPath('data/qas/prompts/desc/instruction.txt'), 'format_file': PosixPath('data/qas/prompts/desc/format.txt'), 'eval_file': PosixPath('data/qas/data/qas_test.jsonl'), 'output_file': PosixPath('results/qa/llama3_instruct/desc/output_test.jsonl'), 'result_file': PosixPath('results/qa/llama3_instruct/desc/result_test.txt')}}


/tmp/ipykernel_1206751/1095863153.py:5: UserWarning:


The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1



Processing model: gpt-4o-mini
{'model_name': 'gpt-4o-mini', 'model_prefix': 'gpt4o_mini', 'precision': 16, 'split': 'test', 'cot': 'desc', 'naming_format': 'name', 'device': 'cuda:0', 'no_wandb': True, 'paths': {'data_root': PosixPath('data'), 'resource_root': PosixPath('qas'), 'result_root': PosixPath('results/qa'), 'instruction_file': PosixPath('data/qas/prompts/desc/instruction.txt'), 'format_file': PosixPath('data/qas/prompts/desc/format.txt'), 'eval_file': PosixPath('data/qas/data/qas_test.jsonl'), 'output_file': PosixPath('results/qa/gpt4o_mini/desc/output_test.jsonl'), 'result_file': PosixPath('results/qa/gpt4o_mini/desc/result_test.txt')}}


...

Explanation: The phrase includes "Mr." which is a male title, and "Mrs." which is a female title, indicating both male and female identifiers. However, since it contains both a male and a female title, it does not clearly indicate a single gender. 
Gender: Neutral
Explanation: The phrase "Paul Armstrong" clearly indicates a male name. "Paul" is a commonly recognized male first name, and "Armstrong" is a surname that does not indicate gender. 
Gender: Male
Explanation: The phrase includes the name "Anthony," which is a male name, and it refers to "Grandfather," a title typically associated with males. Therefore, the phrase clearly indicates a male reference.  
Gender: Male
Explanation: The phrase "Robinson Kreutznaer’s Father" indicates a male figure, as it refers to a father, which is typically a male role. The name "Robinson" is also commonly associated with males. 
Gender: Male
Explanation: The phrase "Jock McChesney" clearly indicates a male name, as "Jock" is a common male given na

/tmp/ipykernel_1206751/1095863153.py:5: UserWarning:


The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1



Explanation: The phrase includes the title "Mr.," which is a clear indicator of a male. 
Gender: Male
Explanation: The phrase "Dorian Gray" refers to a male character from Oscar Wilde's novel "The Picture of Dorian Gray." The name "Dorian" is typically associated with males. 
Gender: Male
Explanation: The phrase "Patesville" does not indicate a specific male or female name or title. It appears to be a place name rather than a personal name. Therefore, it is ambiguous in terms of gender classification. 
Gender: Neutral
Explanation: The phrase "Paul Armstrong" clearly indicates a male name. "Paul" is a commonly recognized male first name, and "Armstrong" is a surname that does not indicate gender. 
Gender: Male
Explanation: The phrase includes the name "Anthony," which is typically a male name, and the title "Grandfather," which also indicates a male figure. Therefore, the phrase clearly indicates a male reference.  
Gender: Male
Explanation: The phrase includes the title "Dr." and the n

/tmp/ipykernel_1206751/1095863153.py:5: UserWarning:


The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1



Explanation: The phrase includes "Step-daughter," which indicates a female relationship, but it also mentions "Brother," which indicates a male relationship. The name "Reginer" is not clearly identifiable as male or female without additional context. Therefore, the overall phrase is ambiguous regarding gender.

Gender: Neutral
Explanation: The phrase "The First Betrothed" does not specify a male or female name or title. It is ambiguous and does not indicate a clear gender association. 
Gender: Neutral
Explanation: The phrase "The Eldest Daughter" clearly indicates a female reference, as it specifically mentions "Daughter," which is a term associated with females. 
Gender: Female
Explanation: The phrase "The Stork" does not indicate a male or female name or title. It is ambiguous and does not specify gender. 
Gender: Neutral
Explanation: The term "The Maiden" typically refers to a young woman or girl, often associated with purity or youth. It clearly indicates a female identity. 
Gender

In [20]:
gender_df = pd.DataFrame(gender_dict).T
gender_df.to_excel("analysis_data/gender/gender_info.xlsx")

In [21]:
gender_unnested_df = pd.DataFrame(gender_unnested_dict).T
gender_unnested_df.to_excel("analysis_data/gender/gender_unnested_info.xlsx")

In [15]:
options_df = pd.DataFrame(options_dict).T
options_df.to_excel("analysis_data/options/options_info.xlsx")

In [16]:
nested_df = pd.DataFrame(nested_dict).T
nested_df.to_excel("analysis_data/nested/nested_info.xlsx")

In [17]:
# ## Convert dict to df and save to summary
accuracy_df = pd.DataFrame(accuracy_dict).T
accuracy_df.to_excel("analysis_data/summary/summary_accuracy.xlsx")

count_df = pd.DataFrame(count_dict).T
count_df.to_excel("analysis_data/summary/summary_count.xlsx")

In [18]:
print(accuracy_dict)

{'mistralai/Mistral-7B-Instruct-v0.3': {'overall': 51.5, 'NOM': 52.5, 'PRON': 50.5, 'fan_NOM': 39.0, 'fan_PRON': 51.67, 'lit_NOM': 66.0, 'lit_PRON': 49.33, 'opts': 57.04, 'noa': 1.67, 'nested': 54.83, 'unnested': 50.12, 'male': 58.04, 'female': 57.1, 'neutral': 53.89, 'pers_pron': 51.35, 'othr_pron': 48.08, 'dialogue': 50.19, 'non_dialogue': 52.38}, 'meta-llama/Meta-Llama-3.1-8B-Instruct': {'overall': 53.33, 'NOM': 53.17, 'PRON': 53.5, 'fan_NOM': 42.33, 'fan_PRON': 55.0, 'lit_NOM': 64.0, 'lit_PRON': 52.0, 'opts': 59.17, 'noa': 0.83, 'nested': 53.69, 'unnested': 53.18, 'male': 62.85, 'female': 60.72, 'neutral': 45.0, 'pers_pron': 51.35, 'othr_pron': 59.62, 'dialogue': 52.71, 'non_dialogue': 58.33}, 'gpt-4o-mini': {'overall': 63.33, 'NOM': 67.67, 'PRON': 59.0, 'fan_NOM': 60.67, 'fan_PRON': 63.33, 'lit_NOM': 74.67, 'lit_PRON': 54.67, 'opts': 63.43, 'noa': 62.5, 'nested': 69.32, 'unnested': 60.85, 'male': 65.06, 'female': 67.41, 'neutral': 50.56, 'pers_pron': 59.68, 'othr_pron': 57.05, 'di

In [19]:
print(accuracy_df.keys())

Index(['overall', 'NOM', 'PRON', 'fan_NOM', 'fan_PRON', 'lit_NOM', 'lit_PRON',
       'opts', 'noa', 'nested', 'unnested', 'male', 'female', 'neutral',
       'pers_pron', 'othr_pron', 'dialogue', 'non_dialogue'],
      dtype='object')


In [30]:
import numpy as np
models = ["gpt4o_mini", "mistral_7b_instruct"]
for model in models:
    accuracy_list = []
    for i in range(10):
        result_file_random = f"results/qa/{model}/base/result_random{i}.txt"
        result_str = open(result_file_random).read()
        accuracy = float(result_str.split("=")[-1])
        accuracy_list.append(accuracy)
    accuracy_model = np.array(accuracy_list)
    print(f"Model: {model}")
    print(accuracy_model)
    print(f"Mean: {accuracy_model.mean()}")
    print(f"Std: {accuracy_model.std()}")

Model: gpt4o_mini
[0.715      0.735      0.72166667 0.67666667 0.67833333 0.72333333
 0.70333333 0.685      0.70666667 0.70333333]
Mean: 0.7048333333333334
Std: 0.018803811670332515
Model: mistral_7b_instruct
[0.67666667 0.64333333 0.64       0.625      0.64833333 0.67833333
 0.66       0.65833333 0.63833333 0.60833333]
Mean: 0.6476666666666667
Std: 0.020672042311618215


In [25]:
print("Instruction:\nRead the text given below. The text has an entity mention marked within \"\"\" {{mention}} (#This is the marked mention) \"\"\". Extract the mention and find who/what the mention refers to in the text.\n\nText:\nChapter 1 It is a truth universally acknowledged , that a single man in possession of a good fortune , must be in want of a wife . However little known the feelings or views of such a man may be on his first entering a neighbourhood , this truth is so well fixed in the minds of the surrounding families , that he is considered the rightful property of some one or other of their daughters . \" My dear Mr. Bennet , \" said his lady to him one day , \" have you heard that Netherfield Park is let at last ? \" Mr. Bennet replied that he had not . \" But it is , \" returned she ; \" for Mrs. Long has just been here , and she told me all about it . \" Mr. Bennet made no answer . \" Do you not want to know who has taken it ? \" cried his wife impatiently . \" _ You _ want to tell me , and I have no objection to hearing it . \" This was invitation enough . \" Why , my dear , you must know , Mrs. Long says that Netherfield is taken by a young man of large fortune from the north of England ; that he came down on Monday in a chaise and four to see the place , and was so much delighted with it , that he agreed with Mr. Morris immediately ; that he is to take possession before Michaelmas , and some of his servants are to be in the house by the end of next week . \" \" What is his name ? \" \" Bingley . \" \" Is he married or single ? \" \" Oh ! Single , my dear , to be sure ! A single man of large fortune ; four or five thousand a year . What a fine thing for our girls ! \" \" How so ? How can it affect them ? \" \" My dear Mr. Bennet , \" replied his wife , \" how can you be so tiresome ! You must know that I am thinking of his marrying one of them . \" \" Is that his design in settling here ? \" \" Design ! Nonsense , how can you talk so ! But it is very likely that he _ may _ fall in love with one of them , and therefore you must visit him as soon as he comes . \" \" I see no occasion for that . You and the girls may go , or you may send them by themselves , which perhaps will be still better , for as you are as handsome as any of them , Mr. Bingley may like you the best of the party . \" \" My dear , you flatter me . I certainly _ have _ had my share of beauty , but I do not pretend to be anything extraordinary now . When a woman has five grown-up daughters , she ought to give over thinking of her own beauty . \" \" In such cases , a woman has not often much beauty to think of . \" \" But , my dear , you must indeed go and see Mr. Bingley when he comes into the neighbourhood . \" \" It is more than I engage for , I assure you . \" \" But consider your daughters . Only think what an establishment it would be for one of them . Sir William and Lady Lucas are determined to go , merely on that account , for in general , you know , they visit no newcomers . Indeed you must go , for it will be impossible for _ us _ to visit him if you do not . \" \" You are over-scrupulous , surely . I dare say Mr. Bingley will be very glad to see you ; and I will send a few lines by you to assure him of my hearty consent to his marrying whichever he chooses of the girls ; though I must throw in a good word for my little Lizzy . \" \" I desire you will do no such thing . Lizzy is not a bit better than the others ; and I am sure she is not half so handsome as Jane , nor half so good-humoured as Lydia . But you are always giving _ her _ the preference . \" \" They have none of them much to recommend them , \" replied he ; \" they are all silly and ignorant like other girls ; but Lizzy has something more of quickness than her sisters . \" \" Mr. Bennet , how _ can _ you abuse your own children in such a way ? You take delight in vexing me . You have no compassion for my poor nerves . \" \" You mistake me , my dear . I have a high respect for your nerves . They are my old friends . I have heard you mention them with consideration these last twenty years at least . \" \" Ah , you do not know what I suffer . \" \" But I hope you will get over it , and live to see many young men of four thousand a year come into the neighbourhood . \" \" It will be no use to us , if twenty such should come , since you will not visit them . \" \" Depend upon it , my dear , that when there are twenty , I will visit them all . \" Mr. Bennet was so odd a mixture of quick parts , sarcastic humour , reserve , and caprice , that the experience of three-and-twenty years had been insufficient to make his wife understand his character . _ Her _ mind was less difficult to develop . She was a woman of mean understanding , little information , and uncertain temper . When she was discontented , she fancied herself nervous . The business of her life was to get her daughters married ; its solace was visiting and news . Chapter 2 Mr. Bennet was among the earliest of those who waited on Mr. Bingley . He had always intended to visit him , though to the last always assuring {{his wife}} (#This is the marked mention) that he should not go ; and till the evening after the visit was paid she had no knowledge of it . It was then disclosed in the following manner . Observing his second daughter employed in trimming a hat , he suddenly addressed her with : \" I hope Mr. Bingley will like it , Lizzy . \" \" We are not in a way to know _ what _ Mr. Bingley likes , \" said her mother resentfully , \" since we are not to visit . \" \" But you forget , mamma , \" said Elizabeth , \" that we shall meet him at the assemblies , and that Mrs. Long promised to introduce him . \" \" I do not believe Mrs. Long will do any such thing . She has two nieces of her own . She is a selfish , hypocritical woman , and I have no opinion of her . \" \" No more have I , \" said Mr. Bennet ; \" and I am glad to find that you do not depend on her serving you . \" Mrs. Bennet deigned not to make any reply , but , unable to contain herself , began scolding one of her daughters . \" Do n't keep coughing so , Kitty , for Heaven 's sake ! Have a little compassion on my nerves . You tear them to pieces . \" \" Kitty has no discretion in her coughs , \" said her father ; \" she times them ill . \" \" I do not cough for my own amusement , \" replied Kitty fretfully . \" When is your next ball to be , Lizzy ? \" \" To-morrow fortnight . \" \" Aye , so it is , \" cried her mother , \" and Mrs. Long does not come back till the day before ; so it will be impossible for her to introduce him , for she will not know him herself . \" \" Then , my dear , you may have the advantage of your friend , and introduce Mr. Bingley to _ her _ . \" \" Impossible , Mr. Bennet , impossible , when I am not acquainted with him myself ; how can you be so teasing ? \" \" I honour your circumspection . A fortnight 's acquaintance is certainly very little . One can not know what a man really is by the end of a fortnight . But if _ we _ do not venture somebody else will ; and after all , Mrs. Long and her neices must stand their chance ; and , therefore , as she will think it an act of kindness , if you decline the office , I will take it on myself . \" The girls stared at their father . Mrs. Bennet said only , \" Nonsense , nonsense ! \" \" What can be the meaning of that emphatic exclamation ? \" cried he . \" Do you consider the forms of introduction , and the stress that is laid on them , as nonsense ? I can not quite agree with you _ there _ . What say you , Mary ? For you are a young lady of deep reflection , I know , and read great books and make extracts . \" Mary wished to say something sensible , but knew not how . \" While Mary is adjusting her ideas , \" he continued , \" let us return to Mr. Bingley . \" \" I am sick of Mr. Bingley , \" cried his wife . \" I am sorry to hear _ that _ ; but why did not you tell me that before ? If I had known as much this morning I certainly would not have called on him . It is very unlucky ; but as I have actually paid the visit , we can not escape the acquaintance now . \" The astonishment of the ladies was just what he wished ; that of Mrs. Bennet perhaps surpassing the rest ; though , when the first tumult of joy was over , she began to declare that it was what she had expected all the while . \" How good it was in you , my dear Mr. Bennet ! But I knew I should persuade you at last . I was sure you loved your girls too well to neglect such an acquaintance . Well , how pleased I am ! and it is such a good joke , too , that you should have gone this morning and never said a word about it till now . \" \" Now , Kitty , you may cough as much as you choose , \" said Mr. Bennet ; and , as he spoke , he left the room , fatigued with the raptures of his wife . \" What an excellent father you have , girls ! \" said she , when the door was shut . \" I do not know how you will ever make him amends for his kindness ; or me , either , for that matter . At our time of life it is not so pleasant , I can tell you , to be making new acquaintances every day ; but for your sakes , we would do anything . Lydia , my love , though you _ are _ the youngest , I dare say Mr. Bingley will dance with you at the next ball . \"\n\nOptions for the answer:\nMary\nKitty\nMrs. Bennet\nMrs. Long\nElizabeth\nMr. Bingley\nMr. Bennet\nLydia\nNetherfield Park\nNone of the Above\n\nNote that you can select the None of the Above option (The mention refers to: None of the Above), if the mention does not refer to any other entity/option. Do not produce any other extra text.\nFollow the below format:\n- Mention:\n- The mention refers to:\n")

Instruction:
Read the text given below. The text has an entity mention marked within """ {{mention}} (#This is the marked mention) """. Extract the mention and find who/what the mention refers to in the text.

Text:
Chapter 1 It is a truth universally acknowledged , that a single man in possession of a good fortune , must be in want of a wife . However little known the feelings or views of such a man may be on his first entering a neighbourhood , this truth is so well fixed in the minds of the surrounding families , that he is considered the rightful property of some one or other of their daughters . " My dear Mr. Bennet , " said his lady to him one day , " have you heard that Netherfield Park is let at last ? " Mr. Bennet replied that he had not . " But it is , " returned she ; " for Mrs. Long has just been here , and she told me all about it . " Mr. Bennet made no answer . " Do you not want to know who has taken it ? " cried his wife impatiently . " _ You _ want to tell me , and I ha

In [29]:
if args.cot == "desc":
    expl_regex = rf"- Explanation: [A-Za-z ,\'\.]{{150,350}}\.\n"
else:
    expl_regex = ""

options = ["Mary", "Kitty", "Mrs. Bennet", "Mrs. Long", "Elizabeth", "Mr. Bingley", "Mr. Bennet", "Lydia", "Netherfield Park", "None of the Above"]
regex_opts = "(" + "|".join(options) + ")"
decoding_regex = rf"- Mention: \{{\{{[A-Za-z ,\'\.]{{1,125}}\}}\}}\n{expl_regex}- The mention refers to: {regex_opts}"
print(decoding_regex)

- Mention: \{\{[A-Za-z ,\'\.]{1,125}\}\}\n- Explanation: [A-Za-z ,\'\.]{150,350}\.\n- The mention refers to: (Mary|Kitty|Mrs. Bennet|Mrs. Long|Elizabeth|Mr. Bingley|Mr. Bennet|Lydia|Netherfield Park|None of the Above)


In [36]:
def get_rejected_docs(dataset_proc, major_entities):
    rejected_doc_list = []
    for doc in dataset_proc:
        entity_id = major_entities[doc]["entity_id"]
        total_mentions = len(dataset_proc[doc]["mentions_vs_stbound"])
        mentions_annotated = len(
            [
                1
                for cluster in dataset_proc[doc]["mentions_vs_clusters"]
                if cluster in entity_id
            ]
        )
        mentions_lost = total_mentions - mentions_annotated
        percent_lost = (mentions_lost / total_mentions) * 100
        num_tokens = len(dataset_proc[doc]["token_vs_tokenstr"])
        if num_tokens < 1000 or percent_lost > 50:
            rejected_doc_list.append(doc)
    return rejected_doc_list

In [38]:
for dataset_name in ["litbank", "fantasy"]:
    dataset_source = dataset_configs[dataset_name][f"train_file"]
    tsv_addr = dataset_configs[dataset_name][f"tsv"]
    doc_me = dataset_configs[dataset_name][f"train_me"]
    dataset = read_jsonl(dataset_source)
    dataset_proc = get_processed_dataset(dataset, tsv_litbank=tsv_addr)
    major_entities = get_major_entities(doc_me)
    rejected_docs = get_rejected_docs(dataset_proc, major_entities_all)
    print(f"Rejected Docs for {dataset_name}: {len(rejected_docs)}")

599_vanity_fair_brat_0
155_the_moonstone_brat_0
2641_a_room_with_a_view_brat_0
1245_night_and_day_brat_0
2891_howards_end_brat_0
2166_king_solomons_mines_brat_0
3457_the_man_of_the_forest_brat_0
932_the_fall_of_the_house_of_usher_brat_0
2084_the_way_of_all_flesh_brat_0
514_little_women_brat_0
120_treasure_island_brat_0
2005_piccadilly_jim_brat_0
27_far_from_the_madding_crowd_brat_0
62_a_princess_of_mars_brat_0
74_the_adventures_of_tom_sawyer_brat_0
36_the_war_of_the_worlds_brat_0
11_alices_adventures_in_wonderland_brat_0
432_the_ambassadors_brat_0
24_o_pioneers_brat_0
32_herland_brat_0
2775_the_good_soldier_brat_0
5348_ragged_dick_or_street_life_in_new_york_with_the_bootblacks_brat_0
9830_the_beautiful_and_damned_brat_0
78_tarzan_of_the_apes_brat_0
1064_the_masque_of_the_red_death_brat_0
209_the_turn_of_the_screw_brat_0
41_the_legend_of_sleepy_hollow_brat_0
1400_great_expectations_brat_0
217_sons_and_lovers_brat_0
3268_the_mysteries_of_udolpho_brat_0
73_the_red_badge_of_courage_an_epis